# Dev New Design Matrix Generator (DMG)

**Goal**: Use this notebook to develop a new, more flexible DMG class.

**Motivation**: The previous version has a "generate_base_matrix" function and then other classes operated on that base matrix (e.g. filtering the history). The structure of the base matrix was not flexible at all and required adding lots of if else logic. Also, adding violation features to a binary model was not possible Now that I am entering a phase of wanting to change features readily, this needs to be updated.

In [ ]:
import pandas as pd
import numpy as np
from multiglm.features.exp_filter import ExpFilter
from multiglm.features.design_matrix_generator import *

from multiglm.data.dataset_loader import *

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = DatasetLoader(animal_ids=["W051"], data_type="new_trained").load_data()

Loading data for animal ids:  ['W051']


In [71]:
config = {
    "bias": (lambda df: dmg.add_bias_column(df)),
    "s_a_norm": (lambda df: dmg.normalize_column(df, "s_a")),
    "s_b_norm": (lambda df: dmg.normalize_column(df, "s_b")),
    "prev_sound_avg": (
        lambda df: dmg.prev_trial_avg(
            df, col_names=["s_a", "s_b"], mask_violations=True, normalize=True
        )
    ),
    "s_a": (lambda df: dmg.copy(df, col_name="s_a")),
    "prev_sa": (
        lambda df: dmg.prev_trial_value(
            df, col_name="s_a", method="map_values", mapping={60: 2}
        )
    ),
}
dmg = DesignMatrixGenerator(df, config)

now set up to make a filtered column. The one thing is this filtered column needs to act on the temp column...hmm. I guess maybe putting it back to the df would have been the right call? but weird copy things were happening (e.g. s_a exists in both). I want to be able to chain things together. So filtering can happen on a column thats already been normalized etc in a separate operations. I want to be able to filter "prev_choice" after I explicitly make prev choice and same for violations. Lets start by writing a method that takes a col, tau and applies the filter to it then we can worry about where it's coming form.

In [70]:
dmg.create()
dmg.X

map_values


,choice,bias,s_a_norm,prev_sa,s_a
0,2,1,-1.649967,0.0,60.0
1,2,1,-1.649967,0.0,60.0
2,2,1,0.019421,0.0,76.0
3,0,1,-0.815273,0.0,68.0
4,1,1,0.854115,68.0,84.0
...,...,...,...,...,...
98918,1,1,1.584472,76.0,91.0
98919,1,1,-1.649967,91.0,60.0
98920,2,1,-0.815273,2.0,68.0
98921,1,1,1.584472,0.0,91.0


In [54]:
sum(dmg.X.prev_sa == 27)

0

In [19]:
x = {"comparison": operator.eq, "value": 2}

x["comparison"]

{'comparison': <function _operator.eq(a, b, /)>, 'value': 2}

In [16]:
col = dmg.X.s_a.copy()
value = 60

import operator

operator.eq(col, 60).astype(int)

0        1
1        1
2        0
3        0
4        0
        ..
98918    0
98919    1
98920    0
98921    0
98922    0
Name: s_a, Length: 98923, dtype: int64